In [1]:
import spotipy
import spotipy.util as util
import pandas as pd
import re

In [202]:
concerts = pd.read_csv("concerts_clean.csv", index_col=0)
concerts['date'] = pd.to_datetime(concerts['date'], format = "%Y/%m/%d")

In [281]:
def get_venue_artists(venue, start_date = "2017-01-01", end_date = "2017-12-31"):
    concert_dates = pd.date_range(start = start_date, end = end_date, freq = 'D')
    artists = []
    for show in concerts.loc[concerts['venue'] == venue, ['date', 'artist']].itertuples():
        if show[1] in concert_dates:
            artists.append(show)
    return artists

In [283]:
#NOTE: I would like to add a way to validate the artist returned by the search here. To do later...
def get_artist_ids(artists):
    sp = spotipy.Spotify()
    artist_plus_ids = []
    for artist in artists:
        search = sp.search(q=artist, type = 'artist', limit = 1)
        try:
            artist_plus_ids.append((search['artists']['items'][0]['name'] ,  search['artists']['items'][0]['id']))
        except IndexError:
            pass
    return artist_plus_ids

In [284]:
def get_venue_artist_ids(venue, start_date = "2017-01-01", end_date = "2017-12-31"):
    sp = spotipy.Spotify()
    artists = get_venue_artists(venue, start_date, end_date)
    artist_plus_ids = []
    for artist in artists:
        search = sp.search(q=artist[2], type = 'artist', limit = 1)
        try:
            artist_plus_ids.append((search['artists']['items'][0]['name'] ,  search['artists']['items'][0]['id']))
        except IndexError:
            pass
    return artist_plus_ids


In [286]:
def create_venue_songlist(venue, start_date = "2017-01-01", end_date = "2017-12-31"):
    sp = spotipy.Spotify()
    songlist = []
    for artist in get_venue_artist_ids(venue, start_date, end_date):
        artist_tracks = sp.artist_top_tracks(artist[1])['tracks']
        if len(artist_tracks) >= 5:
            for track in range(0, 5):
                songlist.append((artist_tracks[track]['name'], \
                                 artist_tracks[track]['id']))
    return songlist
    

In [169]:
def prepare_song_id_list(songlist):
    song_ids = []
    for song in songlist:
        song_ids.append(str(song[1]))
    return song_ids

In [313]:
def create_venue_songlist_ids(venue, start_date = "2017-01-01", end_date = "2017-12-31"):
    songlist = create_venue_songlist(venue, start_date, end_date)
    print type(songlist)
    if len(songlist) >= 100:
        songlist = songlist[0:99]
    ptitle = ("%s from %s to %s") % (venue, start_date, end_date)
    return (ptitle, prepare_song_id_list(songlist))

In [314]:
####Now that we have a list of song IDs, let's authenticate a spotify login, and create the playlist

In [315]:
SPOTIPY_CLIENT_ID = '1723f7534a1c4cde9efe16b0b2753d7f'
SPOTIPY_CLIENT_SECRET = 'e3e4b35b367a430c8b2900ef5d6165a7'
SPOTIPY_REDIRECT_URI = 'http://127.0.0.1:8080/callback/q'

In [329]:
token = util.prompt_for_user_token("", 'playlist-modify-public', SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET, SPOTIPY_REDIRECT_URI)

In [398]:
#set username
username = 'afeierman'
#set venue, start date, end date
venue = "The Echo"
start_date = '2017-03-01'
end_date = '2017-03-15'

#create_venue_sondlist_ids returns a tuple with the playlist title, and a list of Spotify song IDs
playlist_prepped = create_venue_songlist_ids(venue, start_date, end_date)
playlist_title = playlist_prepped[0]
track_ids = playlist_prepped[1]

<type 'list'>


In [399]:
if token:
    sp = spotipy.Spotify(auth=token)
    sp.trace = False #Not sure if this is needed
    sp.user_playlist_create(username, playlist_title)
    playlists = sp.user_playlists(username)
    for playlist in playlists['items']:
        if playlist['name'] == playlist_title:
            playlist_id = playlist['id']
            break
    results = sp.user_playlist_add_tracks(username, playlist_id, track_ids)
    print("Playlist created! %s") % (results)
else:
    print "Authentication failed. Can't get token."

{u'snapshot_id': u'AQK9J0Em9r15UAlO2zzMNBXfXT0QsNxtwctROxImUUhGxidpOsvRaWpTbu33aWu7'}
